# Forecasting de Ventas - Inferencia 2025
Este notebook importa las mismas librerías del entrenamiento y carga el archivo `ventas_2025_inferencia` desde `/data/raw/inferencia` en el DataFrame `df_inferencia`.

## 1. Configurar Rutas y Nombre del Notebook
Definimos variables de ruta para trabajar en VS Code: carpeta de notebooks (`/notebooks`) y carpeta de datos (`/data/raw/inferencia`), y el nombre del archivo de inferencia (`ventas_2025_inferencia`).

In [1]:
# Paths para VS Code (Windows)
from pathlib import Path
base_dir = Path(r"c:\\Users\\dangmoz\\Forecasting_Ventas")
notebooks_dir = base_dir / 'notebooks'
data_inferencia_dir = base_dir / 'data' / 'raw' / 'inferencia'
archivo_inferencia = 'ventas_2025_inferencia.csv'
ruta_inferencia = data_inferencia_dir / archivo_inferencia
print('Ruta notebooks:', notebooks_dir)
print('Ruta datos inferencia:', data_inferencia_dir)
print('Archivo inferencia:', ruta_inferencia)

Ruta notebooks: c:\Users\dangmoz\Forecasting_Ventas\notebooks
Ruta datos inferencia: c:\Users\dangmoz\Forecasting_Ventas\data\raw\inferencia
Archivo inferencia: c:\Users\dangmoz\Forecasting_Ventas\data\raw\inferencia\ventas_2025_inferencia.csv


## 2. Importar Librerías (mismas que el notebook de entrenamiento)
Replicamos los imports utilizados en el notebook de entrenamiento.

In [2]:
# Imports iguales al notebook de entrenamiento
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
print('Imports cargados: pandas, numpy, matplotlib, seaborn, sklearn, streamlit')

Imports cargados: pandas, numpy, matplotlib, seaborn, sklearn, streamlit


## 3. Cargar ventas_2025_inferencia en df_inferencia
Detectamos automáticamente si el archivo es CSV o Parquet y cargamos en `df_inferencia`.

In [3]:
# Carga robusta de inferencia (CSV o Parquet)
import os
df_inferencia = None
ext = ruta_inferencia.suffix.lower()
if ext == '.csv':
    df_inferencia = pd.read_csv(ruta_inferencia, encoding='utf-8')
elif ext == '.parquet':
    df_inferencia = pd.read_parquet(ruta_inferencia)
else:
    raise ValueError(f'Formato no soportado: {ext}. Use .csv o .parquet')
print('Inferencia cargada:', ruta_inferencia)

Inferencia cargada: c:\Users\dangmoz\Forecasting_Ventas\data\raw\inferencia\ventas_2025_inferencia.csv


## 4. Validar Datos Cargados
Mostramos shape, primeras filas, tipos de datos y nulos para asegurar compatibilidad con el pipeline de inferencia.

In [4]:
# Validaciones básicas del df_inferencia
print('Shape:', df_inferencia.shape)
display(df_inferencia.head())
print('\nTipos de datos:')
print(df_inferencia.dtypes)
print('\nNulos por columna:')
print(df_inferencia.isnull().sum())

Shape: (888, 13)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41



Tipos de datos:
fecha                 object
producto_id           object
nombre                object
categoria             object
subcategoria          object
precio_base            int64
es_estrella             bool
unidades_vendidas    float64
precio_venta         float64
ingresos             float64
Amazon               float64
Decathlon            float64
Deporvillage         float64
dtype: object

Nulos por columna:
fecha                  0
producto_id            0
nombre                 0
categoria              0
subcategoria           0
precio_base            0
es_estrella            0
unidades_vendidas    720
precio_venta           0
ingresos             720
Amazon                 0
Decathlon              0
Deporvillage           0
dtype: int64


In [5]:
# ...existing code...
# Validaciones básicas del df_inferencia
print('Shape:', df_inferencia.shape)
display(df_inferencia.head())
print('\nTipos de datos:')
print(df_inferencia.dtypes)
print('\nNulos por columna:')
print(df_inferencia.isnull().sum())
# ...existing code...

# === 5. Transformaciones idénticas a Entrenamiento para df_inferencia ===
import pandas as pd
import numpy as np
from pathlib import Path
import json

# Asegurar tipos y columnas mínimas esperadas
# Se espera al menos: 'fecha', 'producto_id', 'precio_venta', 'precio_base'
# Competencia opcional: 'Amazon', 'Decathlon', 'Deporvillage'
# Ventas/target como referencia (no se usa para inferencia): 'unidades_vendidas', 'ingresos'
if 'fecha' not in df_inferencia.columns:
    raise ValueError("df_inferencia debe contener la columna 'fecha'")
# Parseo de fecha
df_inferencia['fecha'] = pd.to_datetime(df_inferencia['fecha'])

# ===== Calendario y variables temporales (ver Entrenamiento.ipynb secciones 2065-2116) =====
# Año, mes, día, día de semana, nombre de día, semana ISO
df_inferencia['año'] = df_inferencia['fecha'].dt.year
df_inferencia['mes'] = df_inferencia['fecha'].dt.month
df_inferencia['dia_mes'] = df_inferencia['fecha'].dt.day
df_inferencia['dia_semana'] = df_inferencia['fecha'].dt.dayofweek
df_inferencia['nombre_dia_semana'] = df_inferencia['fecha'].dt.day_name()
df_inferencia['semana_año'] = df_inferencia['fecha'].dt.isocalendar().week

# Trimestre, fin de semana
df_inferencia['trimestre'] = df_inferencia['fecha'].dt.quarter
df_inferencia['es_fin_semana'] = df_inferencia['dia_semana'].isin([5, 6])

# Festivos España usando holidays (igual que entrenamiento)
try:
    import holidays
    from datetime import timedelta
    festivos_es = holidays.country_holidays('ES', years=df_inferencia['año'].unique())
    df_inferencia['es_festivo'] = df_inferencia['fecha'].isin(festivos_es)
except Exception:
    # Si no está disponible holidays, marcar False para evitar fallo
    df_inferencia['es_festivo'] = False

# Black Friday: último viernes de noviembre del año
def es_black_friday(fecha: pd.Timestamp) -> bool:
    nov = fecha.replace(month=11, day=1)
    dias_nov = [nov + pd.Timedelta(days=i) for i in range(0, 30)]
    viernes_nov = [d for d in dias_nov if d.weekday() == 4]
    ultimo_viernes = max(viernes_nov) if viernes_nov else None
    return bool(ultimo_viernes and fecha == ultimo_viernes)

df_inferencia['es_black_friday'] = df_inferencia['fecha'].apply(es_black_friday)

# Inicio/fin de mes y trimestre, estaciones, navidad (24-26 dic)
df_inferencia['es_fin_mes'] = df_inferencia['fecha'] == (df_inferencia['fecha'] + pd.offsets.MonthEnd(0))
df_inferencia['es_inicio_trimestre'] = df_inferencia['mes'].isin([1, 4, 7, 10]) & (df_inferencia['dia_mes'] == 1)
df_inferencia['es_fin_trimestre'] = df_inferencia['mes'].isin([3, 6, 9, 12]) & (df_inferencia['fecha'] == (df_inferencia['fecha'] + pd.offsets.MonthEnd(0)))
df_inferencia['es_verano'] = df_inferencia['mes'].isin([6, 7, 8])
df_inferencia['es_navidad'] = df_inferencia['mes'].eq(12) & df_inferencia['dia_mes'].isin([24, 25, 26])

# ===== Ingeniería de precio/competencia (ver Entrenamiento.ipynb línea 2532) =====
# descuento_porcentaje = (precio_venta - precio_base) / precio_base * 100
if {'precio_venta', 'precio_base'}.issubset(df_inferencia.columns):
    base_nonzero = df_inferencia['precio_base'].replace(0, np.nan)
    df_inferencia['descuento_porcentaje'] = ((df_inferencia['precio_venta'] - df_inferencia['precio_base']) / base_nonzero) * 100
    df_inferencia['descuento_porcentaje'] = df_inferencia['descuento_porcentaje'].fillna(0.0)
else:
    df_inferencia['descuento_porcentaje'] = 0.0

competidores = ['Amazon', 'Decathlon', 'Deporvillage']
comp_cols_present = [c for c in competidores if c in df_inferencia.columns]
if comp_cols_present:
    df_inferencia['precio_competencia'] = df_inferencia[comp_cols_present].mean(axis=1)
    # ratio_precio = nuestro precio / precio competencia
    denom = df_inferencia['precio_competencia'].replace(0, np.nan)
    if 'precio_venta' in df_inferencia.columns:
        df_inferencia['ratio_precio'] = df_inferencia['precio_venta'] / denom
    else:
        df_inferencia['ratio_precio'] = np.nan
    # Eliminar columnas originales de competidores para alinear con entrenamiento
    df_inferencia = df_inferencia.drop(columns=comp_cols_present)
else:
    # Si no hay columnas de competidores en inferencia, crear columnas vacías usadas en entrenamiento
    df_inferencia['precio_competencia'] = np.nan
    df_inferencia['ratio_precio'] = np.nan

# ===== One-Hot Encoding similar a entrenamiento =====
# En Entrenamiento se aplicó OHE a variables de texto/temporales (se observan columnas *_h_ en la sección 2861).
# Aquí OHE para 'nombre_dia_semana' y, si existen, 'categoria', 'nombre' u otras categóricas no numéricas.
ohe_cols_candidates = []
if 'nombre_dia_semana' in df_inferencia.columns:
    ohe_cols_candidates.append('nombre_dia_semana')
for c in ['categoria', 'nombre']:
    if c in df_inferencia.columns and df_inferencia[c].dtype == 'object':
        ohe_cols_candidates.append(c)

if ohe_cols_candidates:
    dummies = pd.get_dummies(df_inferencia[ohe_cols_candidates], prefix=[f"{c}_h" for c in ohe_cols_candidates], dtype=int)
    df_inferencia = pd.concat([df_inferencia.drop(columns=ohe_cols_candidates), dummies], axis=1)

# ===== (Opcional) Lags básicos si están definidos por producto y fecha =====
# En Entrenamiento se construyó df_nuevo_lags. Para inferencia mensual 2025, si existen columnas de serie,
# podemos crear lags mínimos sobre precio_venta por producto.
if {'producto_id', 'fecha', 'precio_venta'}.issubset(df_inferencia.columns):
    df_inferencia = df_inferencia.sort_values(['producto_id', 'fecha'])
    df_inferencia['precio_venta_lag_1'] = df_inferencia.groupby('producto_id')['precio_venta'].shift(1)
    df_inferencia['precio_venta_lag_7'] = df_inferencia.groupby('producto_id')['precio_venta'].shift(7)
    df_inferencia['precio_venta_lag_14'] = df_inferencia.groupby('producto_id')['precio_venta'].shift(14)
    # Relleno conservador
    for c in ['precio_venta_lag_1', 'precio_venta_lag_7', 'precio_venta_lag_14']:
        df_inferencia[c] = df_inferencia[c].fillna(df_inferencia['precio_venta'])

# ===== Alinear columnas con las usadas en entrenamiento =====
models_dir = Path("../models")
features_full_path = models_dir / "histgb_features_full.json"
if features_full_path.exists():
    with open(features_full_path, "r", encoding="utf-8") as f:
        features_json = json.load(f)
    feature_cols = features_json.get('features', [])
else:
    # Fallback: usar las columnas numéricas no objetivo
    feature_cols = list(df_inferencia.select_dtypes(exclude=['object']).columns)

# Eliminar columnas que no se usan para el modelo (iguales a entrenamiento)
exclude_cols = ['fecha', 'ingresos', 'unidades_vendidas']
df_model = df_inferencia.drop(columns=[c for c in exclude_cols if c in df_inferencia.columns])

# Mantener solo numéricas
df_model = df_model.select_dtypes(exclude=['object'])

# Alinear orden y rellenar faltantes de columnas esperadas por el modelo
for col in feature_cols:
    if col not in df_model.columns:
        # Crear columna faltante con 0 (o NaN y luego permitir imputación del modelo si aplica)
        df_model[col] = 0
# Quitar columnas extra no presentes en el modelo
df_model = df_model[feature_cols]

# ===== Filtrar solo noviembre y eliminar octubre =====
# Mantener el índice con las filas originales para posible trazabilidad
if 'mes' not in df_inferencia.columns:
    df_inferencia['mes'] = df_inferencia['fecha'].dt.month

mask_noviembre = df_inferencia['mes'] == 11
df_inferencia_nov = df_inferencia.loc[mask_noviembre].copy()
df_model_nov = df_model.loc[mask_noviembre].copy()

print(f"Registros originales: {len(df_inferencia)}, noviembre: {len(df_inferencia_nov)}")

# ===== Guardar df_inferencia transformado (estructura completa + columnas del modelo) =====
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Guardamos la versión transformada completa para trazabilidad
out_path = processed_dir / "df_inferencia_transformado.csv"
df_inferencia_nov.to_csv(out_path, index=False, encoding='utf-8')
print(f"df_inferencia transformado (noviembre) guardado en: {out_path}")

# Opcional: mostrar preview
display(df_inferencia_nov.head())
print("Columnas del modelo (noviembre):", len(df_model_nov.columns))
print(df_model_nov.dtypes.head())
# ...existing code...

Shape: (888, 13)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41



Tipos de datos:
fecha                 object
producto_id           object
nombre                object
categoria             object
subcategoria          object
precio_base            int64
es_estrella             bool
unidades_vendidas    float64
precio_venta         float64
ingresos             float64
Amazon               float64
Decathlon            float64
Deporvillage         float64
dtype: object

Nulos por columna:
fecha                  0
producto_id            0
nombre                 0
categoria              0
subcategoria           0
precio_base            0
es_estrella            0
unidades_vendidas    720
precio_venta           0
ingresos             720
Amazon                 0
Decathlon              0
Deporvillage           0
dtype: int64


C:\Users\dangmoz\AppData\Local\Temp\ipykernel_6676\675448645.py:44: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_inferencia['es_festivo'] = df_inferencia['fecha'].isin(festivos_es)


Registros originales: 888, noviembre: 720
df_inferencia transformado (noviembre) guardado en: ..\data\processed\df_inferencia_transformado.csv


,fecha,producto_id,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,año,mes,...,nombre_h_Quechua MH500,nombre_h_Reebok Floatride Energy 5,nombre_h_Reebok Professional Deck,nombre_h_Salomon Speedcross 5 GTX,nombre_h_Sveltus Kettlebell 12kg,nombre_h_The North Face Borealis,nombre_h_Trek Marlin 7,precio_venta_lag_1,precio_venta_lag_7,precio_venta_lag_14
168,2025-11-01,PROD_001,Zapatillas Running,115,True,NaN,115.0,NaN,2025,11,...,0,0,0,0,0,0,0,110.93,113.13,115.0
192,2025-11-02,PROD_001,Zapatillas Running,115,True,NaN,115.0,NaN,2025,11,...,0,0,0,0,0,0,0,115.00,105.75,115.0
216,2025-11-03,PROD_001,Zapatillas Running,115,True,NaN,115.0,NaN,2025,11,...,0,0,0,0,0,0,0,115.00,114.95,115.0
240,2025-11-04,PROD_001,Zapatillas Running,115,True,NaN,115.0,NaN,2025,11,...,0,0,0,0,0,0,0,115.00,117.31,115.0
264,2025-11-05,PROD_001,Zapatillas Running,115,True,NaN,115.0,NaN,2025,11,...,0,0,0,0,0,0,0,115.00,108.10,115.0


Columnas del modelo (noviembre): 118
precio_base       int64
es_estrella        bool
precio_venta    float64
dia_semana        int32
año               int32
dtype: object


In [ ]:
# Cuantos elementos diferentes de producto_id hay en df_inferencia. Cuantos productos son unicos?
df_inferencia.producto_id.nunique()

24

In [ ]:
df_inferencia.columns

Index(['fecha', 'producto_id', 'subcategoria', 'precio_base', 'es_estrella',
       'unidades_vendidas', 'precio_venta', 'ingresos', 'año', 'mes',
       'dia_mes', 'dia_semana', 'semana_año', 'trimestre', 'es_fin_semana',
       'es_festivo', 'es_black_friday', 'es_fin_mes', 'es_inicio_trimestre',
       'es_fin_trimestre', 'es_verano', 'es_navidad', 'descuento_porcentaje',
       'precio_competencia', 'ratio_precio', 'nombre_dia_semana_h_Friday',
       'nombre_dia_semana_h_Monday', 'nombre_dia_semana_h_Saturday',
       'nombre_dia_semana_h_Sunday', 'nombre_dia_semana_h_Thursday',
       'nombre_dia_semana_h_Tuesday', 'nombre_dia_semana_h_Wednesday',
       'categoria_h_Fitness', 'categoria_h_Outdoor', 'categoria_h_Running',
       'categoria_h_Wellness', 'nombre_h_Adidas Own The Run Jacket',
       'nombre_h_Adidas Ultraboost 23', 'nombre_h_Asics Gel Nimbus 25',
       'nombre_h_Bowflex SelectTech 552', 'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas S

: 

In [8]:
# Cuantos elementos diferentes de fecha hay en df_inferencia
df_inferencia.fecha.nunique()

37

In [10]:
# Cantidad de registros en df_inferencia
df_inferencia.shape

(888, 63)

## 5. Conteo de valores nulos en noviembre
Calculamos cuántos valores nulos hay en el DataFrame de inferencia filtrado a noviembre y el detalle por columna.

In [11]:
# Conteo de nulos para el mes de noviembre
import pandas as pd
import numpy as np

# Usar df_inferencia_nov si ya fue creado; si no, filtrarlo ahora
if 'df_inferencia_nov' in globals():
    df_nov = df_inferencia_nov.copy()
else:
    if 'df_inferencia' not in globals():
        raise NameError('df_inferencia no está disponible. Ejecuta las celdas de carga primero.')
    if 'fecha' not in df_inferencia.columns:
        raise KeyError("La columna 'fecha' no existe en df_inferencia.")
    # Asegurar tipo datetime y filtrar noviembre
    df_tmp = df_inferencia.copy()
    if not pd.api.types.is_datetime64_any_dtype(df_tmp['fecha']):
        df_tmp['fecha'] = pd.to_datetime(df_tmp['fecha'])
    df_tmp['mes'] = df_tmp['fecha'].dt.month
    df_nov = df_tmp[df_tmp['mes'] == 11].copy()

total_nulos = int(df_nov.isnull().sum().sum())
nulos_por_col = df_nov.isnull().sum().sort_values(ascending=False)
columnas_con_nulos = int((nulos_por_col > 0).sum())
filas_con_alguna_na = int(df_nov.isnull().any(axis=1).sum())

print(f'Registros en noviembre: {len(df_nov)}')
print(f'Total de valores nulos en noviembre: {total_nulos}')
print(f'Columnas con al menos un nulo: {columnas_con_nulos}')
print(f'Filas con al menos un nulo: {filas_con_alguna_na}')

# Mostrar detalle de columnas con nulos (si existen)
nulos_detalle = nulos_por_col[nulos_por_col > 0]
if not nulos_detalle.empty:
    display(nulos_detalle.to_frame('nulos'))
else:
    print('No se encontraron columnas con nulos en noviembre.')

Registros en noviembre: 720
Total de valores nulos en noviembre: 1440
Columnas con al menos un nulo: 2
Filas con al menos un nulo: 720


,nulos
unidades_vendidas,720
ingresos,720
